In [7]:
import numpy as np
import pandas as pd
from matplotlib import rc
rc("animation", html="jshtml")

import importlib
import models
importlib.reload(models)
import tools
importlib.reload(tools)
import tools.play_viz as play_viz
from tools import *
from models import (
    KinematicModel,
    train_eval_model,
    KinematicBoundaryModel,
    BayesianKinematicModel,
    HierarchicalBayesianKinematicModel,
    BayesianKinematicHMM,
    fit_model_up_to_week,
    train_eval_until_week,
)


In [2]:
def get_game_play_index(input_path):
    df = pd.read_csv(input_path, usecols=["game_id", "play_id"])
    df = df.drop_duplicates().sort_values(["game_id", "play_id"])
    return df


In [ ]:
week = 6 
index_df = get_game_play_index(f"train/input_2023_w{week:02d}.csv")
print(f"{index_df.shape[0]} plays for week {week}")
idx = 7
gi = index_df.iloc[idx].game_id
pi = index_df.iloc[idx].play_id
print(f"Selected: game_id={gi}, play_id={pi}")


793 plays for week 6
Selected: game_id=2023101200, play_id=589


# Hierarchical Bayesian Kinematic Model

This model extends the basic Bayesian kinematic model by adding hierarchical structure:
- Position-specific noise scales (WRs, RBs, CBs may have different movement patterns)
- Player-specific parameters (nested in position)
- Global priors that share information across all players/positions


In [ ]:
hierarchical_model = HierarchicalBayesianKinematicModel(fps=10.0, use_accel=True)
hierarchical_model = fit_model_up_to_week(
    hierarchical_model,
    max_train_week=1,
    source="input",   
    use_position_hierarchy=True,
    use_player_hierarchy=True, 
    max_samples=20000, 
    draws=50,  
    tune=50,   
    target_accept=0.9,
    chains=1   
)


[fit_model_up_to_week] Training on weeks: [1]


Building step df (input):   0%|          | 0/1 [00:00<?, ?week/s]

Only 50 samples per chain. Reliable r-hat and ESS diagnostics require longer chains for accurate estimate.
Initializing NUTS using jitter+adapt_diag...


[fit_model_up_to_week] Fitting model 'hierarchical_bayes_kinematic' on 275,625 rows...
[HierarchicalBayesianKinematicModel] Subsampling from 275,625 to 20,000 rows for faster training


Sequential sampling (1 chains in 1 job)
NUTS: [sigma_x_global, sigma_y_global]


Output()

Sampling 1 chain for 50 tune and 50 draw iterations (50 + 50 draws total) took 326 seconds.
The number of samples is too small to check convergence reliably.


[fit_model_up_to_week] Done fitting 'hierarchical_bayes_kinematic'.


In [17]:
# Visualize with heatmaps showing probability density
importlib.reload(play_viz)

ani_hierarchical = visualize_predictions(
    hierarchical_model,
    week=week,
    game_id=gi,
    play_id=pi,
    horizon=3,
    interval=120,
    pause_time=None,
    bayes_samples=300,  # More samples for better heatmap
    show_paths=False,
    show_cones=False,
    show_heatmaps=True,  # Enable heatmaps!
    heatmap_alpha=1,
    heatmap_grid_resolution=60,
    heatmap_bandwidth=0.8,
    show_legend=True
)
ani_hierarchical


In [18]:
from tools.play_viz import save_animation
save_animation(ani_hierarchical, "demo.html", fps=10)

Saving animation to demo.html (format: .html, fps: 10, writer: html)...
✓ Animation saved successfully to demo.html
  File size: 0.01 MB
